### 파일 읽어오기
- txt 형태의 파일 로드하기

In [1]:
import logging
from logging.config import dictConfig

dictConfig({
    'version': 1,
    'formatters': {
        'default': {
            'format': '[%(asctime)s] %(message)s',
        }
    },
    'handlers': {
        'file': {
            'level': 'DEBUG',
            'class': 'logging.FileHandler',
            'filename': 'debug.log',
            'formatter': 'default',
        },
    },
    'root': {
        'level': 'DEBUG',
        'handlers': ['file']
    }
})


def myfunc():
    logging.debug("함수가 시작되었습니다.")


myfunc()

In [2]:
def make_logger(name=None):
    #1 logger instance를 만든다.
    logger = logging.getLogger(name)

    #2 logger의 level을 가장 낮은 수준인 DEBUG로 설정해둔다.
    logger.setLevel(logging.DEBUG)

    #3 formatter 지정
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    #4 handler instance 생성
    console = logging.StreamHandler()
    file_handler = logging.FileHandler(filename="test.log")
    
    #5 handler 별로 다른 level 설정
    console.setLevel(logging.INFO)
    file_handler.setLevel(logging.DEBUG)

    #6 handler 출력 format 지정
    console.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    #7 logger에 handler 추가
    logger.addHandler(console)
    logger.addHandler(file_handler)

    return logger

In [3]:
make_logger()

<RootLogger root (DEBUG)>

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm 
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

In [5]:
# txt 파일 불러오기
train_df = pd.read_table('1. 실습용자료.txt',sep='|',encoding="cp949")
test_df = pd.read_table('2. 모델개발용자료.txt',sep='|',encoding="cp949")

In [6]:
# csv 파일 형태로 추출
train_df.to_csv("1. 실습용자료.csv",encoding="utf-8-sig",index=False)
test_df.to_csv("2. 모델개발용자료.csv",encoding="utf-8-sig",index=False)

In [7]:
# csv 파일 불러오기 / 컬럼명 조정
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]

test_df = pd.read_csv("2. 모델개발용자료.csv")
test_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]

### 1차 텍스트 전처리

In [8]:
# 공백으로 처리할것들
#del_special_lst = ['５', '∼', '>', '）', 'Ⅱ', '／', '．', '"', ']', '-', ',', '～',
#       '＇', '㎜', '_', '６', '３', '（', '@', '}', '１', '<', '→', '₂', 'Ⅲ',
#       '=', '，', '&', '↘', '/', '·', '＆', '#', 'ㆍ', ' ', ':', '^', '`',
#       '%', 'º', '①',"'"]
#np.save("del_special_lst.npy",del_special_lst)

In [9]:
def new_data_cleansing_text(df,col_lst):
    for col in col_lst:

        # null값 채우기
        df[col] = df[col].fillna("")
        
        # 양쪽 공백 삭제
        df[col] = df[col].apply(lambda x : x.strip())
        
        # 특정 특수문자 삭제
        except_lst = ["無","地","金","銅","茶","外","內","\'",";","℃"]
        for ex in except_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,'', x))
            
        #영어 문자 사이의 특수기호 삭제
        df[col] = df[col].str.replace(pat="(?<=[a-z])[/$](?=[a-z])",repl = "",regex=True)
        
        # 특수기호 영어 원래 영어로 변경
        eng_lst = ["Ｖ","Ｇ","Ｐ","Ｌ","Ｔ"]
        eng = ["V","G","P","L","T"]
        for i in range(len(eng_lst)):
            df[col] = df[col].apply(lambda x : re.sub(eng_lst[i],eng[i], x))

        # 숫자 제거
        df[col] = df[col].apply(lambda x : re.sub(r'[0-9]', '', x))
    
        # 면적 관련 기호 처리
        replace_lst = ["㎠","㎣","㎡","㎥","m²","㎢","㎟","²"]
        for ex in replace_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,'면적', x))
        
        # 공백으로 처리할것들
        del_special_lst = ['５', '∼', '>', '）', 'Ⅱ', '／', '．', '"', ']', '-', ',', '～',
       '＇', '㎜', '_', '６', '３', '（', '@', '}', '１', '<', '→', '₂', 'Ⅲ',
       '=', '，', '&', '↘', '/', '·', '＆', '#', 'ㆍ', ' ', ':', '^', '`',
       '%', 'º', '①',"'"]
        for ex in del_special_lst:
            df[col] = df[col].apply(lambda x : re.sub(ex,' ', x))

        fail = ['\\', '+', ')', '(', '*', '['] # 공백으로 처리할 것들2
        
        df[col] = df[col].apply(lambda x : re.sub(r"[+]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[*]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[\\]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[(]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[)]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[[]"," ",x))
        df[col] = df[col].apply(lambda x : re.sub(r"[]]"," ",x))
        
        df[col] = df[col].apply(lambda x : x.replace("."," "))
        
        # 다중공백 -> 단일공백
        df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
        # 삭제 안하고 남겨둘 특수문자
        no_except_lst = ["!","~","？","?"] 
        df[col] = df[col].apply(lambda x : re.sub('？', '?', x))
    
        # 한글 자음모음 삭제
        df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
        
        #양쪽 공백삭제
        df[col] = df[col].apply(lambda x : x.strip())
        
    return df

In [10]:
# train_df와 test_df 결합
set_df = pd.concat([train_df,test_df],axis=0).reset_index(drop=True)
set_df.shape

(1100000, 7)

In [11]:
# 1차 전처리
set_df_clean = new_data_cleansing_text(set_df.copy(),["text_obj","text_mthd","text_deal"])
set_df_clean["text_sum"] = set_df_clean["text_obj"] +" "+ set_df_clean["text_mthd"] +" "+ set_df_clean["text_deal"]

2022-04-12 11:50:04,575 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [12]:
# 1차 전처리한 데이터 분할
train_set_df_clean = set_df_clean.iloc[:1000000,:].reset_index(drop=True)
test_set_df_clean = set_df_clean.iloc[1000000:,:].reset_index(drop=True)

In [13]:
# column 데이터 타입 변경 int -> str
train_set_df_clean = train_set_df_clean.astype({'중분류': 'int',
                          '소분류': 'int'})

train_set_df_clean = train_set_df_clean.astype({'중분류': 'str',
                          '소분류': 'str'})

In [14]:
#label 결합
train_set_df_clean["label"] = ( train_set_df_clean["대분류"] + train_set_df_clean["중분류"]
                               + train_set_df_clean["소분류"] )


# 결합 이후 drop
for col in ["대분류","중분류","소분류"]:
    train_set_df_clean.drop(col,axis=1,inplace=True)
    test_set_df_clean.drop(col,axis=1,inplace=True)

In [15]:
# column명 변경
train_set_df_clean.rename(columns={'text_sum':'document'},inplace=True)
test_set_df_clean.rename(columns={'text_sum':'document'},inplace=True)

In [16]:
# 1차 전처리한 데이터 저장
train_set_df_clean.to_csv("train_preprocessed_ver2_no_lower.csv",encoding="utf-8-sig",index=False)
test_set_df_clean.to_csv("test_preprocessed_ver2_no_lower.csv",encoding="utf-8-sig",index=False)

### 2차 텍스트 전처리
- ?가 존재하는 항목에 대한 전처리

In [17]:
train_ver2 = pd.read_csv("train_preprocessed_ver2_no_lower.csv")
test_ver2 = pd.read_csv("test_preprocessed_ver2_no_lower.csv")

In [18]:
# 물음표 존재하는 항목들
train_ver2_ques = train_ver2[train_ver2["document"].str.contains("\?")]
test_ver2_ques = test_ver2[test_ver2["document"].str.contains("\?")]

In [19]:
# 데이터 개수 확인
print(len(train_ver2_ques))
print(len(test_ver2_ques))

2416
275


In [20]:
# 텍스트 전처리 1차
def ques(df,col):
    
    df[col] = df[col].apply(lambda x : x.strip())
    
    df[col] = df[col].apply(lambda x : x.replace(r"커피?","커피"))
    df[col] = df[col].apply(lambda x : x.replace(r"중고?","중고"))
    df[col] = df[col].apply(lambda x : x.replace(r"플라스틱?","플라스틱"))
    df[col] = df[col].apply(lambda x : x.replace(r"케?","케잌"))
    df[col] = df[col].apply(lambda x : x.replace(r"?퓨터","컴퓨터"))
    df[col] = df[col].apply(lambda x : x.replace(r"?볶이","떡볶이"))
    df[col] = df[col].apply(lambda x : x.replace(r"떡?이","떡볶이"))
    df[col] = df[col].apply(lambda x : x.replace(r"?뢰","의뢰"))
    df[col] = df[col].apply(lambda x : x.replace(r"?가루","밀가루"))
    df[col] = df[col].apply(lambda x : x.replace(r"?단구입","원단구입"))
    df[col] = df[col].apply(lambda x : x.replace(r"?료기기","의료기기"))
    df[col] = df[col].apply(lambda x : x.replace(r"법?상담","법률상담"))
    df[col] = df[col].apply(lambda x : x.replace(r"접?시설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"접객?설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"접객시?","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"맛사지?","맛사지샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"?톱미용","손톱미용"))
    df[col] = df[col].apply(lambda x : x.replace(r"마사지?","마사지샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"마사지 ?","마사지샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"수?교육","수학교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"수학?육","수학교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"수학교?","수학교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"고?","고객"))
    df[col] = df[col].apply(lambda x : x.replace(r"?원","학원"))
    df[col] = df[col].apply(lambda x : x.replace(r"네일?","네일샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"네일아트 ?","네일아트샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"네일아트?","네일아트샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"슈퍼마?","슈퍼마켓"))
    df[col] = df[col].apply(lambda x : x.replace(r"?매","판매"))
    df[col] = df[col].apply(lambda x : x.replace(r"?반인","일반인"))
    df[col] = df[col].apply(lambda x : x.replace(r"피부?","피부샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"?션","펜션"))
    df[col] = df[col].apply(lambda x : x.replace(r"내일?","네일샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"네일 ?","네일샵"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"뷰티?","뷰티샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"위?받아","위탁받아"))
    df[col] = df[col].apply(lambda x : x.replace(r"선교?동","선교활동"))
    df[col] = df[col].apply(lambda x : x.replace(r"에?","에게"))
    df[col] = df[col].apply(lambda x : x.replace(r"체?단련","체력단련"))
    df[col] = df[col].apply(lambda x : x.replace(r"납?","납품"))
    df[col] = df[col].apply(lambda x : x.replace(r"수수료?고","수수료 받고"))
    df[col] = df[col].apply(lambda x : x.replace(r"수수료르?고","수수료를 받고"))
    df[col] = df[col].apply(lambda x : x.replace(r"치?판매","치킨 판매"))
    df[col] = df[col].apply(lambda x : x.replace(r"?프 빨래","셀프 빨래"))
    df[col] = df[col].apply(lambda x : x.replace(r"의뢰?아","의뢰받아"))
    df[col] = df[col].apply(lambda x : x.replace(r"세?소","세탁소"))
    df[col] = df[col].apply(lambda x : x.replace(r"선교?동","선교활동"))
    df[col] = df[col].apply(lambda x : x.replace(r"고객의요?","고객의 요청"))
    df[col] = df[col].apply(lambda x : x.replace(r"고객의 요?","고객의 요청"))
    df[col] = df[col].apply(lambda x : x.replace(r"차?운전자","차량운전자"))
    df[col] = df[col].apply(lambda x : x.replace(r"전자상거?","전자상거래"))
    df[col] = df[col].apply(lambda x : x.replace(r"의뢰를?아","의뢰를 받아"))
    df[col] = df[col].apply(lambda x : x.replace(r"예?","예배"))
    df[col] = df[col].apply(lambda x : x.replace(r"?추고","갖추고"))
    df[col] = df[col].apply(lambda x : x.replace(r"영어?육","영어교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"영어 ?육","영어교육"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"?박서비스","숙박서비스"))
    df[col] = df[col].apply(lambda x : x.replace(r"주유시?","주유시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"금속?","금속"))
    df[col] = df[col].apply(lambda x : x.replace(r"기타 ?","기타"))
    df[col] = df[col].apply(lambda x : x.replace(r"단독주?","단독주택"))
    df[col] = df[col].apply(lambda x : x.replace(r"?반고객","일반고객"))
    df[col] = df[col].apply(lambda x : x.replace(r"일?소비자","일반소비자"))
    df[col] = df[col].apply(lambda x : x.replace(r"애견미용?","애견 미용샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"애견 미용?","애견 미용샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"직조?","직조"))
    df[col] = df[col].apply(lambda x : x.replace(r"애견?","애견샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"접?설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"광고간?","광고간행"))
    df[col] = df[col].apply(lambda x : x.replace(r"식?에서","식당에서"))
    df[col] = df[col].apply(lambda x : x.replace(r"사무?","사무실"))
    df[col] = df[col].apply(lambda x : x.replace(r"중학?","중학생"))
    df[col] = df[col].apply(lambda x : x.replace(r"독서?","독서실"))
    df[col] = df[col].apply(lambda x : x.replace(r"네?샵","네일샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"절연?","절연"))
    df[col] = df[col].apply(lambda x : x.replace(r"?고","절연"))
    df[col] = df[col].apply(lambda x : x.replace(r"?구장","당구장"))
    df[col] = df[col].apply(lambda x : x.replace(r"?라스틱","플라스틱"))
    df[col] = df[col].apply(lambda x : x.replace(r"?가게에서","꽃가게에서"))
    df[col] = df[col].apply(lambda x : x.replace(r"?가계에서","꽃가게에서"))
    df[col] = df[col].apply(lambda x : x.replace(r"가?에서","가게에서"))
    df[col] = df[col].apply(lambda x : x.replace(r"엔지니어링 ?","엔지니어링"))
    df[col] = df[col].apply(lambda x : x.replace(r"고객요?","고객요구"))
    df[col] = df[col].apply(lambda x : x.replace(r"고객?구","고객요구"))
    df[col] = df[col].apply(lambda x : x.replace(r"제조업 ?","제조업"))
    df[col] = df[col].apply(lambda x : x.replace(r"광고매?","광고매체"))
    df[col] = df[col].apply(lambda x : x.replace(r"기계?장비","기계장비"))
    df[col] = df[col].apply(lambda x : x.replace(r"맛사지 ?","마사지샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"?반소비자","일반소비자"))
    df[col] = df[col].apply(lambda x : x.replace(r"미용?","미용실"))
    df[col] = df[col].apply(lambda x : x.replace(r"미?실","미용실"))
    df[col] = df[col].apply(lambda x : x.replace(r"연구개발업 ?","연구개발업"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"부동산?개소","부동산중개소"))
    df[col] = df[col].apply(lambda x : x.replace(r"?복이","떡볶이"))
    df[col] = df[col].apply(lambda x : x.replace(r"?복기","떡볶이"))
    df[col] = df[col].apply(lambda x : x.replace(r"?볶기","떡볶이"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"인력?급","인력공급"))
    df[col] = df[col].apply(lambda x : x.replace(r"인력공급?","인력공급"))
    df[col] = df[col].apply(lambda x : x.replace(r"제조업 ?","제조업"))
    df[col] = df[col].apply(lambda x : x.replace(r"제조?","제조업"))
    df[col] = df[col].apply(lambda x : x.replace(r"접객시설?","접객시설을 "))
    df[col] = df[col].apply(lambda x : x.replace(r"잡?시설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"접?시설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"접객요원?","접객요원"))
    df[col] = df[col].apply(lambda x : x.replace(r"적객요원?","접객요원"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"제조업 평?","제조업 평"))
    df[col] = df[col].apply(lambda x : x.replace(r"일반차?","일반차량"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"피부관리?","피부관리샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"피부미용?","피부미용샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"?생","학생"))
    df[col] = df[col].apply(lambda x : x.replace(r"교?","교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"?육","교육"))
    df[col] = df[col].apply(lambda x : x.replace(r"초등?생","초등학생"))
    df[col] = df[col].apply(lambda x : x.replace(r"초등?을","초등학생을"))
    df[col] = df[col].apply(lambda x : x.replace(r"헤어?","헤어샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"플라스틱 ?","플라스틱"))
    df[col] = df[col].apply(lambda x : x.replace(r"플라스?","플라스틱"))
    df[col] = df[col].apply(lambda x : x.replace(r"헤어?","헤어샵"))
    df[col] = df[col].apply(lambda x : x.replace(r"송풍기및배기?","송풍기 및 배기장치"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"계약?중개","계약 및 중개"))
    df[col] = df[col].apply(lambda x : x.replace(r"계약?g중개","계약 및 중개"))
    df[col] = df[col].apply(lambda x : x.replace(r"계약및중개","계약 및 중개"))

    df[col] = df[col].apply(lambda x : x.replace(r"인가?아","인가받아"))
    df[col] = df[col].apply(lambda x : x.replace(r"음?점","음식점"))
    df[col] = df[col].apply(lambda x : x.replace(r"?객시설","접객시설"))
    df[col] = df[col].apply(lambda x : x.replace(r"의?를받아","의뢰를 받아"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"기타 일?","기타 일반"))
    df[col] = df[col].apply(lambda x : x.replace(r"제조업 전?","제조업 전"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"접객요원및무도시설?이","접객요원 및 무도시설 없이"))
    df[col] = df[col].apply(lambda x : x.replace(r"접객요원무도시설없이","접객요원 및 무도시설 없이"))
    df[col] = df[col].apply(lambda x : x.replace(r"학?","학생"))
    df[col] = df[col].apply(lambda x : x.replace(r"학생들 대상으?","학생들 대상으로"))
    df[col] = df[col].apply(lambda x : x.replace(r"학생을대상으로","학생들 대상으로"))
    df[col] = df[col].apply(lambda x : x.replace(r"학생들대상으로","학생들 대상으로"))
    df[col] = df[col].apply(lambda x : x.replace(r"학생을데상으로","학생들 대상으로"))
    df[col] = df[col].apply(lambda x : x.replace(r"계약?G중개","계약 중개"))
    
    return df

In [21]:
# ? 전처리 적용
train_ver2_ques = ques(train_ver2_ques,"document")
test_ver2_ques = ques(test_ver2_ques,"document")

In [22]:
# 물음표 존재하는 항목들
train_ver2_ques_left = train_ver2_ques[train_ver2_ques["document"].str.contains("\?")]
test_ver2_ques_left = test_ver2_ques[test_ver2_ques["document"].str.contains("\?")]

# ? 전처리 후 데이터 개수 확인
print(len(train_ver2_ques_left))
print(len(test_ver2_ques_left))

758
124


전처리 되지 않은 항목에 대해서는 csv 파일을 보며 직접 전처리 및 교정

In [23]:
# ? 전처리 후 나머지 항목 직접 교정한 데이터
train_ver2_ques_left = pd.read_csv("left_train.csv")
test_ver2_ques_left = pd.read_csv("left_test.csv")

In [24]:
# ? 2차 전처리
def ques_2nd(df,col):
    
    df[col] = df[col].apply(lambda x : x.strip())
    
    df[col] = df[col].apply(lambda x : x.replace(r"미용원","미용실"))
    df[col] = df[col].apply(lambda x : x.replace(r"미장원","미용실"))
    df[col] = df[col].apply(lambda x : x.replace(r"이용원","미용실"))
    
    df[col] = df[col].apply(lambda x : x.replace(r"마삿지","마사지"))
    df[col] = df[col].apply(lambda x : x.replace(r"샵","숍"))
    df[col] = df[col].apply(lambda x : x.replace(r"맛사지","마사지"))
    
    return df

In [25]:
train_ver2_ques_left = ques_2nd(train_ver2_ques_left,"document")
test_ver2_ques_left = ques_2nd(test_ver2_ques_left,"document")

### 데이터 전처리 최종결합

학습용

In [26]:
train_ver2 = pd.read_csv("train_preprocessed_ver2_no_lower.csv")
test_ver2 = pd.read_csv("test_preprocessed_ver2_no_lower.csv")

In [27]:
# 전처리 항목들 결합
concat1 = pd.concat([train_ver2_ques,
                     train_ver2_ques_left], axis=0)

In [28]:
concat1 = concat1.drop_duplicates(["AI_id"],keep="last").sort_values("AI_id").reset_index(drop=True)
concat2 = pd.concat([train_ver2,
                     concat1], axis=0)
concat2 = concat2.drop_duplicates(["AI_id"],keep="last").sort_values("AI_id").reset_index(drop=True)

In [29]:
concat2.to_csv("train_preprocessed_ver2_no_lower_no_ques.csv",encoding="utf-8-sig",index=False)

평가용

In [30]:
concat1 = pd.concat([test_ver2_ques,
                     test_ver2_ques_left], axis=0)

In [31]:
concat1 = concat1.drop_duplicates(["AI_id"],keep="last").sort_values("AI_id").reset_index(drop=True)
concat2 = pd.concat([test_ver2,
                     concat1], axis=0)
concat2 = concat2.drop_duplicates(["AI_id"],keep="last").sort_values("AI_id").reset_index(drop=True)

In [32]:
concat2.to_csv("test_preprocessed_ver2_no_lower_no_ques.csv",encoding="utf-8-sig",index=False)